In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [2]:
os.getcwd()

'c:\\Users\\hyo05\\Documents\\Portfolio\\Apple transaction'

In [15]:
# file and DB URL
db_url = "C:/Users/hyo05/Documents/Portfolio/Apple transaction/csv/"
file_url = db_url + "Apple Card Transactions - April 2021.csv"
print(file_url)

C:/Users/hyo05/Documents/Portfolio/Apple transaction/csv/Apple Card Transactions - April 2021.csv


In [17]:
# Look for the similarity in the file name for loop

In [16]:
# Read and display data
april = pd.read_csv(file_url)
pd.set_option("max_rows", 10)
display(april)

,Transaction Date,Clearing Date,Description,Merchant,Category,Type,Amount (USD)
0,04/30/2021,05/01/2021,APPLE.COM/BILL ONE APPLE PARK WAY 866-712-7753...,Apple Services,Other,Purchase,2.99
1,04/29/2021,04/30/2021,APPLE.COM/BILL ONE APPLE PARK WAY 866-712-7753...,Apple Services,Other,Purchase,4.99
2,04/29/2021,04/30/2021,APPLE.COM/BILL ONE APPLE PARK WAY 866-712-7753...,Apple Services,Other,Purchase,19.99
3,04/29/2021,04/30/2021,SQ *SEOUL HOTDOG 1822 ARCH STREET BERKELEY 947...,Seoul Hotdog,Restaurants,Purchase,10.90
4,04/29/2021,04/29/2021,PAPA JOHN'S #4764 5412 SAN PABLO AVE STE B 510...,Papa John's,Restaurants,Purchase,21.84
...,...,...,...,...,...,...,...
36,04/03/2021,04/05/2021,APPLE.COM/BILL ONE APPLE PARK WAY 866-712-7753...,Apple Services,Other,Purchase,3.99
37,04/04/2021,04/04/2021,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,Ach Deposit Internet Transfer From Account End...,Payment,Payment,-343.30
38,04/02/2021,04/02/2021,SOUNDCLOUD INC 510 TREAT AVE SAN FRANCISCO9411...,Soundcloud,Utilities,Purchase,4.99
39,04/01/2021,04/02/2021,GEICO *AUTO ONE GEICO PLAZA 800-841-3000 20076...,Geico,Insurance,Purchase,264.66
